In [1]:
import cv2
import os
import random
import numpy as np

from register_dataset import *
from detectron2.engine import DefaultPredictor, default_setup
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo

from utils import draw_rect

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
dataset_name_train = "HRSIDtrain2017"
dataset_name_test = "HRSIDtest2017"
dataset_dicts = DatasetCatalog.get(dataset_name_test)
dataset_metadata = MetadataCatalog.get(dataset_name_test)

# Load model

In [3]:
def setup(config_file):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml") 
    cfg.merge_from_file(config_file)
    default_setup(cfg, None)
    return cfg

In [4]:
# cfg already contains everything we've set previously. Now we changed it a little bit for inference:
cfg = setup(config_file='yamls/HRSID/Faster-RCNN-R50.yaml' )
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1   # custom testing threshold for R-CNN
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.1   
predictor = DefaultPredictor(cfg)

[02/28 20:16:27 detectron2]: Rank of current process: 0. World size: 1
[02/28 20:16:28 detectron2]: Environment info:
----------------------  -------------------------------------------------------------------
sys.platform            linux
Python                  3.7.3 (default, Mar 27 2019, 22:11:17) [GCC 7.3.0]
numpy                   1.18.5
detectron2              0.3 @/home/jtli/SAR/detectron2/detectron2
Compiler                GCC 8.3
CUDA compiler           CUDA 10.1
detectron2 arch flags   7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.6.0 @/home/jtli/anaconda3/lib/python3.7/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0,1                 GeForce RTX 2080 Ti (arch=7.5)
CUDA_HOME               /usr/local/cuda
Pillow                  7.2.0
torchvision             0.7.0 @/home/jtli/anaconda3/lib/python3.7/site-packages/torchvision
torchvision arch flags  3.5, 5.0, 6.0, 7.0, 7.5
fvcore                  0.1.2.post20201211
cv2    

# Visualization

In [5]:
save_path = "../exp_figs/Faster-RCNN-R50"
!mkdir -p $save_path

In [8]:
for i, d in enumerate(dataset_dicts):
    if not i in [50, 338, 1389, 1587]: continue
    im = cv2.imread(d["file_name"])
    name = os.path.split(d["file_name"])[1]
    
    true_coords = [ann['bbox'] for ann in d['annotations']]
    true_coords = np.array(true_coords, dtype='int')  
    
    outputs = predictor(im)
    pred_coords = outputs['instances'].get_fields()['pred_boxes'].tensor.cpu().numpy().astype('int')
    pred_coords[:,2] -= pred_coords[:,0]
    pred_coords[:,3] -= pred_coords[:,1]    
    draw_rect(im, true_coords, pred_coords, save_path=os.path.join(save_path, str(i)))
